In [1]:
import numpy
import numpy as np
import pandas as pd
import sys, glob
import cdsapi, zipfile, os, time
import warnings
import shutil
import xarray
from datetime import date
warnings.filterwarnings('ignore')
sys.path.append(os.getcwd()+'/../cds-backend/code/')
import cds_eua4 as eua

import ray
import h5py
import json

ray.init(num_cpus=20)

{'node_ip_address': '131.130.157.11',
 'raylet_ip_address': '131.130.157.11',
 'redis_address': '131.130.157.11:6379',
 'object_store_address': '/tmp/ray/session_2022-06-21_16-08-26_955197_3159503/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2022-06-21_16-08-26_955197_3159503/sockets/raylet',
 'webui_url': None,
 'session_dir': '/tmp/ray/session_2022-06-21_16-08-26_955197_3159503',
 'metrics_export_port': 45653,
 'gcs_address': '131.130.157.11:44385',
 'node_id': '4fc0d3adf816561fb8131b634cb50fa8302272008ca5e4b42b060297'}

In [2]:
eua

<module 'cds_eua4' from '/mnt/users/staff/a1400070/CEUAS/CEUAS/public/testing/../cds-backend/code/cds_eua4.py'>

In [3]:
@ray.remote
def f(sid):
    
    try:
        with h5py.File(sid, 'r') as of:
            if 'advanced_homogenisation/latitude_displacement' in of:
                if 'recordindices/126' in of:
                    lat = of['observations_table/latitude'][-1]
                    lon = of['observations_table/longitude'][-1]
                    avlascs = of['recordindices/126'].shape[0]
                    sname = ''
                    for i in of['header_table/station_name'][0]:
                        sname += (i.decode())
                    statid = sid.split('/')[-1].split('_C')[0]
                    if (1.92 <= lat <= 71.84) and ((315.86-360) <= lon <= 60.21):
                        sys.path.append(os.getcwd()+'/../cds-backend/code/')
                        import cds_eua4 as eua
                        indf = eua.CDMDataset(sid).to_dataframe(groups=['advanced_homogenisation', 'observations_table'], 
                                                                variables=['latitude_displacement', 'longitude_displacement',
                                                                           'time_since_launch', 'z_coordinate', 'date_time']
                                                               )
                        indf = indf[indf.date_time.dt.year > 1980]
                        indf = indf[indf.z_coordinate < 10000]
                        indf = indf.dropna()
                        if len(indf) > 1:
                            minlatd = np.nanmin(indf.latitude_displacement)
                            maxlatd = np.nanmax(indf.latitude_displacement)
                            minlond = np.nanmin(indf.longitude_displacement)
                            maxlond = np.nanmax(indf.longitude_displacement)
                            maxasct = np.nanmax(indf.time_since_launch)

                            return [sname, statid, avlascs, lat, lon, minlatd, maxlatd, minlond, maxlond, maxasct]
    except:
        pass
    return np.nan
            

In [4]:
# Start 4 tasks in parallel.
files = glob.glob('/users/staff/a1400070/scratch/converted_v8/*.nc')
result_ids = []
for i in files[:]:
#     print(f(i))
    result_ids.append(f.remote(i))
results = ray.get(result_ids)

ray.shutdown()

In [5]:
type(results)

list

In [6]:
out = []
do = {}
do['name'] = []
do['statid'] = []
do['ascents'] = []
do['lat'] = []
do['lon'] = []
do['min_lat_d'] = []
do['max_lat_d'] = []
do['min_lon_d'] = []
do['max_lon_d'] = []
do['max_time'] = []
for i in results:
    if not isinstance(i, float):
        do['name'].append(i[0])
        do['statid'].append(i[1])
        do['ascents'].append(i[2])
        do['lat'].append(i[3])
        do['lon'].append(i[4])
        do['min_lat_d'].append(i[5])
        do['max_lat_d'].append(i[6])
        do['min_lon_d'].append(i[7])
        do['max_lon_d'].append(i[8])
        do['max_time'].append(i[9])

In [8]:
# do

In [9]:
testdf = pd.DataFrame.from_dict(do)

In [10]:
strpd_df = testdf.dropna()

In [11]:
strpd_df

,name,statid,ascents,lat,lon,min_lat_d,max_lat_d,min_lon_d,max_lon_d,max_time
0,BODO VI,0-20000-0-01152,39318,67.285103,14.454200,-0.316405,0.074579,-0.057933,0.660207,2026.041655
1,NAIMAKKA,0-20000-0-02055,4,68.680000,21.530001,-0.019103,0.003133,-0.003271,0.234782,1166.088065
2,LULEA-KALLAX (2185-0),0-20000-0-02185,44027,65.543312,22.114531,-0.000351,0.017154,-0.005499,0.045096,396.679511
3,SUNDSVALL HARNOSAND (2365-0),0-20000-0-02365,57238,62.531700,17.436100,-0.003721,0.000000,0.000000,0.005257,107.296170
4,VISBY AEROLOGISKA STATION,0-20000-0-02591,43231,57.670830,18.345310,-0.011138,0.008686,-0.028541,0.010024,388.948975
...,...,...,...,...,...,...,...,...,...,...
225,KUWAIT INTERNATIONAL AIRPORT (40582-1),0-20001-0-40582,48224,29.240000,47.970001,-0.263667,0.127403,-0.001365,0.529186,2126.874865
226,DOUALA R.S.,0-20001-0-64910,36629,4.020000,9.700000,-0.031161,0.012040,-0.063162,0.013645,1189.665534
227,HEMSBY,0-20300-0-03496,70383,52.680000,1.680000,-0.473522,0.037547,-0.537145,0.200629,3838.744511
228,GRACIOSA/AERODROMO,0-620-2001-0507,1591,39.090000,-28.030001,-0.070422,0.073157,0.000000,0.320259,1528.051028


In [12]:
strpd_df.to_csv('euro_station_displacement_range.csv')